In [1]:
import numpy as np
import galapy as gp

In [2]:
class GXY () :
    
    def __init__ ( self ) :
        
        self.sfh = gp.StarFormationHistory.SFH()
        self.csp = gp.CompositeStellarPopulation.CSP()
        self.ism = gp.InterStellarMedium.ISM()
        

In [3]:
gxy = GXY()

In [1]:
import numpy as np
import galapy as gp

In [4]:

def gen_params_dict ( tau_quench = 2.e+10, model = 'insitu' ) :

    _models = {
        # In-Situ SF model
        'insitu' : {
            'psi_max' :  100.,
            'tau_star' : 3.e+8,
        },
        # Constant SF model
        'constant' : {
            'psi' : 1.,
        },
        # Delayed-Exponential model
        'delayedexp' : {
            'psi_norm' : 1.,
            'k_shape' :  0.2,
            'tau_star' : 1.e+8,
        },
        # Log-Normal model
        'lognormal' : {
            'psi_norm' :   100.,
            'sigma_star' : 2.,
            'tau_star' :   3.e+8,
        }
    }

    return { 'tau_quench' : tau_quench, 'model' : model }.update( **_models[ model ] )

In [8]:
tau_quench = 2.e+10
model = 'insitu'
_models = {
    # In-Situ SF model
    'insitu' : {
        'psi_max' :  100.,
        'tau_star' : 3.e+8,
    },
    # Constant SF model
    'constant' : {
        'psi' : 1.,
    },
    # Delayed-Exponential model
    'delayedexp' : {
        'psi_norm' : 1.,
        'k_shape' :  0.2,
        'tau_star' : 1.e+8,
    },
    # Log-Normal model
    'lognormal' : {
        'psi_norm' :   100.,
        'sigma_star' : 2.,
        'tau_star' :   3.e+8,
    }
}
out = { 'tau_quench' : tau_quench, 'model' : model }

In [10]:
out.update( _models[model] )

In [11]:
out

{'tau_quench': 20000000000.0,
 'model': 'insitu',
 'psi_max': 100.0,
 'tau_star': 300000000.0}

In [5]:
gen_params_dict()

In [2]:
sfh_model_choice = {
    # In-Situ SF model
    'insitu' : {
        'psi_max' :  100.,
        'tau_star' : 3.e+8,
    },
    # Constant SF model
    'constant' : {
        'psi' : 1.,
    },
    # Delayed-Exponential model
    'delayedexp' : {
        'psi_norm' : 1.,
        'k_shape' :  0.2,
        'tau_star' : 1.e+8,
    },
    # Log-Normal model
    'lognormal' : {
        'psi_norm' :   100.,
        'sigma_star' : 2.,
        'tau_star' :   3.e+8,
    }
}

In [3]:
from collections.abc import MutableMapping as MM

def rec_update (dd, **kwargs) :
    for k, v in kwargs.items() :
        if isinstance(v,MM) :
            rec_update( dd[k], **v )
        elif k in dd.keys() :
            dd[k] = v
        else :
            print( f'Key {k} not valid' )
            continue
    return;

def build_param_dict ( **kwargs ) :
    out = {
        'age' : 1.e+9,
        'sfh' : { 
            'tau_quench' : 1.e+8,
            'model' : 'insitu',
        },
        'csp' : {
            'ssp_library' : gp.SSP_LIB['bc03.basel.chab.extend']            
        },
        'ism' : {
            'dd' : {
                'f_MC'  : 0.5,
                'norm'  : 1.,
                'Mdust' : 3.55988e+07,
                'Rdust' : 1000.,
                'f_PAH' : 0.2
            },
            'mc' : {
                'f_MC'    : 0.5,
                'norm'    : 100.,
                'N_MC'    : 8.5e+03,
                'R_MC'    : 16,
                'Zgas'    : 0.466,
                'tau_esc' : 1.e+7,
                'Mgas'    : 8.51101e+09,
            },
        },
    }
         
    rec_update( out, **kwargs )
    out['sfh']['params'] = sfh_model_choice[ out['sfh']['model'] ]
    
    return out

In [4]:
def rec_return ( dd, keylist ) :
    if len(keylist) > 1 and isinstance( dd, MM ) :
        return rec_return( dd[keylist.pop(0)], keylist )
    return dd[keylist.pop(0)]

In [5]:
def trap_int ( xx, yy ) :
    xx = np.asarray(xx)
    yy = np.asarray(yy)
    b1b2 = yy[:-1] + yy[1:]
    hh = xx[1:] - xx[:-1]
    return np.sum( 0.5 * b1b2 * hh )

In [6]:
Lsun = 3.828e+33

In [7]:
class Galaxy () :
    import galapy.SFH_core as csfh
    import galapy.CSP_core as ccsp
    import galapy.ISM_core as cism
    
    def __init__ ( self, **kwargs ) :
        import galapy.internal.CPySFH as csfh
        import galapy.internal.CPyCSP as ccsp
        import galapy.internal.CPyISM as cism
        self.rcparams = ( build_param_dict( **kwargs ) )
        self.sfh = csfh.CSFH( self['sfh.tau_quench'], self['sfh.model'] )
        self.wavel, self.times, self.metals, Lgrid = ccsp.loadSSP(self['csp.ssp_library']) 
        self.csp = ccsp.CCSP( self.wavel, self.times, 
                              self.metals, Lgrid )
        class ISM () :
            def __init__ (self) :
                self.dd = cism.CDD()
                self.mc = cism.CMC()
        self.ism = ISM()
        
        self.set_params()
        
    def __setitem__ ( self, key, value ) :
        rec_update( self.rcparams, **{ key : value } )
        return;
    def __getitem__ ( self, key ) :
        return rec_return( self.rcparams, key.split('.') )
    
    def set_params ( self, **kwargs ) :
        self.rcparams.update( kwargs )
        self.sfh.set_params( np.asarray(list(self['sfh.params'].values())) )
        self.csp.set_params( *self.sfh.time_grid( self['age'], self.times, self.metals ) )
        self.ism.dd.set_params(np.asarray(list(self['ism.dd'].values())))
        self.ism.mc.set_params(np.asarray(list(self['ism.mc'].values())))
        return;
    
    def SED ( self, lidx, **kwargs ) :
        self.set_params( **kwargs )
        lsub = self.wavel[lidx]
        attDD = self.ism.dd.attenuation(lsub)
        attMC = self.ism.mc.attenuation(lsub)
        etaMC = self.ism.mc.eta(self.times)
        attTotMC = 1 - ( 1 - attMC )[:,np.newaxis] * etaMC[np.newaxis,:]
        attTot = attTotMC * attDD[:,np.newaxis]
        Lunatt = self.csp.emission( lidx )
        LattMC = self.csp.emission( lidx, attTotMC )
        Latt = self.csp.emission( lidx, attTot )
        Tdd = self.ism.dd.temperature( 0.5 * Lsun * trap_int( lsub, ( LattMC - Latt ) ) )
        Tmc = self.ism.mc.temperature( 0.5 * Lsun * trap_int( lsub, ( Lunatt - LattMC ) ) )
        Ldd = self.ism.dd.emission(lsub)
        Lmc = self.ism.mc.emission(lsub)
        return Latt + Ldd + Lmc

In [8]:
%time gxy = Galaxy()

CPU times: user 10.5 ms, sys: 58 ms, total: 68.6 ms
Wall time: 67.9 ms


In [71]:
%time gxy.set_params(**build_param_dict())

CPU times: user 46 µs, sys: 31 µs, total: 77 µs
Wall time: 80.6 µs


In [9]:
subidx = np.arange(91,len(gxy.wavel),100,dtype=np.uint64)

In [15]:
%time Lum = gxy.SED( subidx )

CPU times: user 713 µs, sys: 710 µs, total: 1.42 ms
Wall time: 1.24 ms
